#### 获取可能的选点

In [ ]:
def actions(board):
    """返回当前board的所有可能的子树.将棋盘中所有距离已有棋子2格之内的点加入actions中.

    """
    actions = set()
    for i in range(4,19): 
        for j in range(4,19):
            if board[i][j] == 0 or board[i][j] == 1:
                for k in range(max(4,i-1),min(i+2,19)):
                    for l in range(max(4,j-1),min(j+2,19)):
                        if board[k][l] == 'Y':
                            actions.add((k,l))
    return actions

#### $minimax$算法 +$ Alpha-Beta $ 剪枝

In [ ]:
def alphabeta(board,depth,alpha,beta,color:int,computercolor:int): # 人工智能走子
    """Alpha-Beta Pruning
    """

    if depth == 0:
        A = evalBoard(board,computercolor)
        return A.get_score()

    if color == computercolor: # 当前是电脑方

        maxEval=ninf
        trimmedactions = trim_actions(board,actions(board),computercolor)

        for action in trimmedactions:

            tmpboard = copy.deepcopy(board)
            tmpboard[action[0]][action[1]] = color

            # 特殊情况，赢了
            if win(tmpboard,action[0],action[1]):
                if depth == 3:
                    return action
                else:
                    return 10000

            evaluate = alphabeta(tmpboard,depth-1,alpha,beta,int(not color),computercolor) # 将position的child赋给eval。传参时，处理的子树会获知[已处理子树的根节点的取值信息]。
            tmp = maxEval
            maxEval = max(evaluate,maxEval) 
            if maxEval > tmp and depth == 3: # 如果当前节点的值比最大值大，则更新最优选择
                bestAct = action
            alpha = max(alpha,evaluate) # 一棵子树清理完毕，就更新一次alpha。
            if beta <= alpha: # 如果在某个节点处，对方的最小值小于我方最大，那么对面肯定不会选这一支（因为传的alphabeta值>=alpha）,剪掉这一action.
                break
        if depth == 3: # 如果是最大深度，则返回最优选择
            print ('Maximum score for the computer is %d' % maxEval)
            return bestAct
        else: # 否则继续搜索
            return maxEval

    else:

        minEval=pinf
        trimmedactions = trim_actions(board,actions(board),computercolor)

        for action in trimmedactions:

            tmpboard = copy.deepcopy(board)
            tmpboard[action[0]][action[1]] = color

            if win(tmpboard,action[0],action[1]):
                return -10000

            evaluate = alphabeta(tmpboard,depth-1,alpha,beta,int(not color),computercolor)
            minEval = min(evaluate,minEval)
            beta = min(beta,evaluate)
            if beta <= alpha:
                break
        return minEval

#### 局势评估

In [ ]:
class evalBoard():
    """评估棋盘的分数

    """
    def __init__(self,chesslist:list,color:int):
        self.chesslist = chesslist
        self.x = str(color)
        self.y = str(int(not color))
        self.score = 0
        self.potential = 0
        self.bcf = [0] # list是可变的，此处利用作为 “引用类型”
        self.wcf = [0]
        self.bif = [0]
        self.wif = [0]
        self.blf = [0]
        self.wlf = [0]
        self.wdf = [0]
        self.blt = [0]
        self.wlt = [0]
        self.bst = [0]
        self.wst = [0]
        self.tuple_dict = {
            "111113": self.bcf,       # 黑棋连5

            "000003": self.wcf,       # 白棋连5

            "01111Y": self.bif,       # 黑棋连4
            "111Y10": self.bif,
            "11Y110": self.bif,
            "1Y1110": self.bif,
            "Y11110": self.bif,    # 黑棋冲四

            "10000Y": self.wif,   
            "000Y01": self.wif,
            "00Y001": self.wif,
            "0Y0001": self.wif,
            "Y00001": self.wif,    # 白棋冲四

            "Y1111Y": self.blf,   # 黑棋活四

            "Y0000Y": self.wlf,   # 白棋活四

            "100001": self.wdf,   # 白棋死四
            "N00001": self.wdf,

            "Y111Y3": self.blt,   # 黑棋活三

            "Y000Y3": self.wlt ,  # 白棋活三

            "Y11103": self.bst,   # 黑棋眠三

            "Y00013": self.wst   # 白棋眠三
        }
            
 
    def match_tuple(self,Tup:str):
        Tup = Tup.replace(self.y,"0")
        Tup = Tup.replace(self.x,"1")
        Tup = Tup.replace("False","0")
        Tup = Tup.replace("True","1")
        if Tup in self.tuple_dict:
            self.tuple_dict[Tup][0] += 1
        else:
            Tup[5] = 3
            if Tup in self.tuple_dict:
                self.tuple_dict[Tup][0] += 1

    def get_score(self):
        """
        黑棋连5,评分为10000
        白棋连5,评分为 -10000
        黑棋两个冲四可以当成一个活四
        白棋有活四，评分为 -9050
        白棋有冲四，评分为 -9040
        黑棋有活四，评分为 9030
        黑棋有冲四和活三，评分为 9020
        黑棋没有冲四，且白棋有活三，评分为 -9010
        黑棋有2个活三, 且白棋没有活三,评分为 9000
        下面针对黑棋或白棋的活三，眠三，活二，眠二的个数依次增加分数，评分为（黑棋得分 - 白棋得分）
        """
        # 分别计算横、竖、左下、右下四个方向的五元组
        directions = [[1,0],[1,1],[0,1],[-1,1],[-1,0],[-1,-1],[0,-1],[1,-1]]
        for i in range(4,19):
            for j in range(4,19):
                for direction in directions:
                    try:
                        elem = ""
                        for k in range(6):
                            pos = np.array([i,j])+np.array(direction)*k
                            elem += str(self.chesslist[pos[0]][pos[1]]) 
                        self.match_tuple(elem)
                    except: # 越界
                        continue

        if self.bcf[0] > 0: # 黑棋连5，赢
            self.score = 10000
        elif self.wcf[0] > 0 or self.wlf[0] > 0: # 白棋连5，输
            self.score = -10000
        elif self.wlf[0] > 0: # 白棋活4，输
            self.score = -9050
        elif self.wif[0] > 0: # 白棋冲四，输
            self.score = -9040
        elif self.bif[0] > 1 or self.blf[0] > 0:
            self.score = 9030
        elif self.blf[0] > 0 and self.blt[0] > 0:
            self.score = 9020
        elif self.wdf[0] > 0: # 白棋死四，惩罚
            self.score = -10
        elif self.wlt[0] > 0:
            self.score = -9010
        elif self.blt[0] > 0 and self.wlt[0] == 0:
            self.score = 9000
        return self.score